## Read in data

path:

/xdisk/chanc/share/

there are two folders: GRMHD_dt1000M and GRMHD_dt10M

under GRMHD_dt1000M: (5000-5900, step=100) <br>Ma+0.5_w5 <br>Ma+0.94_w5<br> Ma-0.5_w5<br> Ma-0.94_w5<br> Ma0_w5<br> Sa+0.5_w5<br> Sa+0.94_w5<br> Sa-0.5_w5<br> Sa-0.94_w5<br> Sa0_w5

under GRMHD_dt10M: (5000 - 5999, step=1) <br>Ma+0.94_w5<br>Ma-0.94_w5 <br> Sa+0.94_w5

USE GRMHD_dt1000M

In [1]:
from toolz import valmap

import h5py
import pyharm

from matplotlib   import pyplot as plt
import numpy as np
from pyharm.plots import plot_dumps as pplt

In [2]:
i0 = 5000 # initial  frame
i1 = 5900 # 1 + last frame

fmt = 'torus.out0.{:05d}.h5'

qualities = ['RHO', 'Pg', 'bsq', 'beta', 'Theta', 'Thetae']

In [3]:
def fname(i, src): # closure on `src` and `fmt`
    return (src+fmt).format(i)

In [4]:
path_1000M = '/xdisk/chanc/share/GRMHD_dt1000M/'
src_ma = ['Ma+0.5_w5', 'Ma+0.94_w5', 'Ma-0.5_w5', 'Ma-0.94_w5', 'Ma0_w5']
src_sa = ['Sa+0.5_w5', 'Sa+0.94_w5', 'Sa-0.5_w5', 'Sa-0.94_w5', 'Sa0_w5']
src_ma_1000M = []
src_sa_1000M = []
for i in range(len(src_ma)):
    src_ma_1000M.append(path_1000M+src_ma[i]+'/')
    src_sa_1000M.append(path_1000M+src_sa[i]+'/')

dump0 = pyharm.load_dump(fname(i0, src_ma_1000M[0]))
R_low = 1.
R_high = [1, 10, 40, 160]
# first row of SANE model params

In [5]:
def get_dumps(i0, i1, src, step=1):
    dumps = {}
    for i in range(i0, i1, step):
        dumps[str(i)] = pyharm.load_dump(fname(i, src))
    return dumps

In [6]:
# some constants
cl = 29979245800.0 # speed of light in cgs
mp = 1.67262171e-24
kb = 1.3806505e-16

In [7]:
# R
def R(dump, Rh, Rl):
    beta = dump['beta']
    return Rl/(1+beta**2)+Rh*beta**2/(1+beta**2)

In [8]:
# electron temperature
def Te(dump, Rh, Rl, gamma):
    if gamma == 4/3:
        y = 1
        z = 3
        gamma_i = 4/3
        gamma_e = 7/3
    if gamma == 13/9:
        y = 1
        z = 2
        gamma_i = 13/9
        gamma_e = 133/36
    if gamma == 5/3:
        y = 1
        z = 1
        gamma_i = 5/3
        gamma_e = 5/3
    beta = dump['beta']
    u = dump['Pg']/(gamma-1)
    r = R(dump, Rh, Rl)
    numerator = mp*cl**2*u*(gamma_e-1)*(gamma_i-1)
    denom = kb*dump['RHO']*(y*(gamma_i-1)+r*z*(gamma_e-1))
    Te = numerator/denom
    dump.cache['Te'] = Te
    return Te

Use dictionary

look for γ: https://arxiv.org/pdf/2202.11721.pdf <br> Assume ideal gas -> $Γ = \frac{5}{3} + \frac{4}{3}(1 - x_{e})$ where $x_{e}$ is the electron fraction. <br> $\gamma = \frac{4}{3}$, $x_{e} = \frac{1}{4}$, so ions:electrons = 3:1 (y=1, z=3; $\gamma_{i}=\gamma=\frac{4}{3}, \gamma_e = \gamma + x_{e} / (1 - x_{e})(5/3 - \gamma) = \frac{7}{3}$) <br> $\gamma = \frac{13}{9}$, $x_{e} = \frac{1}{3}$, so ions:electrons = 2:1 (y=1, z=2; $\gamma_{i}=\gamma=\frac{13}{9}, \gamma_e = \gamma + x_{e} / (1 - x_{e})(5/3 - \gamma) = \frac{133}{36}$) <br> $\gamma = \gamma_{i}=\frac{5}{3}$, so the gas is fully composed of ions (i.e. $\gamma_{e} = 0$) (? y=z=1; $\gamma_{i}=\gamma_{e} = \gamma = \frac{5}{3}$) <br>

In [9]:
gammas = [4/3, 13/9, 5/3]


## Model